In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from tensorboardX import SummaryWriter
from IPython.display import clear_output


/home/henrytsui/anaconda3/envs/IDS/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BERT_PATH = 'bert-base-uncased'
MAX_LEN = 64
DEVICE = "cuda"
BZ = 128
WD = 1e-9
LR = 1e-5
EPS = 100
TMAX = 15
COMMENT = f"lr{LR}-B{BZ}-EPS{EPS}-TOU{TMAX}"

In [3]:
merge = pd.read_csv("../data/merge.csv", index_col=0)
merge = shuffle(merge)
ds_size = merge.shape[0]

In [4]:
class Stock(Dataset):
    def __init__(self, df) -> None:
        self.df = df
        self.score = [float(score) for score in df["score"]]
        self.tokenizer = BertTokenizer.from_pretrained(BERT_PATH)
        self.content = [self.tokenizer(text,padding='max_length', 
                       max_length = MAX_LEN, 
                       truncation=True,
                       return_tensors="pt") for text in df["content"]]

    def __getitem__(self, idx):
        return self.score[idx], self.content[idx]

    def __len__(self):
        return self.df.shape[0]

In [5]:
t0, t1, t2 = np.split(merge.sample(frac=1, random_state=42), [int(.8*ds_size), int(.9*ds_size)])
dataset = {x: Stock(s) for x, s in [("train", t0), ("valid", t1), ("test", t2)]}
loader = {x: DataLoader(dataset[x], batch_size=BZ, num_workers=24, shuffle=True) 
                                            for x in ["train", "valid", "test"]}

In [6]:
for state in ["train", "valid", "test"]:
    sz = 0
    for num, t in loader[state]:
        sz += len(num)
    print(sz)

20222
2528
2528


In [7]:
class Bert4price(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.bert = BertModel.from_pretrained(BERT_PATH)
        self.fc = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.Linear(128, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    def forward(self, text, mask):
        _, output = self.bert(input_ids=text, attention_mask=mask,return_dict=False)
        output = self.fc(output)
        return torch.squeeze(output, 1).double()

In [8]:
bert = Bert4price()
bert = bert.to(DEVICE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
crierion = nn.MSELoss()
optimizer = torch.optim.Adam(bert.parameters(), lr = LR, weight_decay=WD)
lr_sch = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=TMAX)

In [10]:
writer = SummaryWriter(comment=COMMENT)
for epoch in range(EPS):
    for state in ["train", "valid"]:
        clear_output(wait=True)
        tqdm_bar = tqdm(loader[state])
        tqdm_bar.set_description(f"[{epoch+1}/{EPS}]")
        for value, content in tqdm_bar:
            
            text, mask = content["input_ids"].squeeze(1), content["attention_mask"]
            text, mask = text.to(DEVICE), mask.to(DEVICE)
        
            value = value.to(DEVICE)
            output = bert(text, mask)
            loss = crierion(output, value)
            if state == "train":
                optimizer.zero_grad() 
                loss.backward()
                optimizer.step()
                lr_sch.step()
            writer.add_scalar(f"{state}-loss", loss, epoch)
    else:
        vnum = value.detach().cpu().numpy()
        onum = output.detach().cpu().numpy()
        result = np.array([vnum, onum]).T
        valid_result = pd.DataFrame(result, columns = ['ans','output'])
        valid_result.to_csv("../data/result.csv")

[1/100]:   0%|          | 0/158 [00:00<?, ?it/s]